In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline
import itertools
import time
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold

In [3]:
data = pd.read_csv('data/Hitters.csv').dropna()
data

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,497,127,7,65,48,37,5,2703,806,32,379,311,138,N,E,325,9,3,700.0,N
318,492,136,5,76,50,94,12,5511,1511,39,897,451,875,A,E,313,381,20,875.0,A
319,475,126,3,61,43,52,6,1700,433,7,217,93,146,A,W,37,113,7,385.0,A
320,573,144,9,85,60,78,8,3198,857,97,470,420,332,A,E,1314,131,12,960.0,A


In [4]:
df = pd.get_dummies(data, columns=['League', 'Division', 'NewLeague'], drop_first=True)
df

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary,League_N,Division_W,NewLeague_N
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,632,43,10,475.0,1,1,1
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,880,82,14,480.0,0,1,0
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,200,11,3,500.0,1,0,1
4,321,87,10,39,42,30,2,396,101,12,48,46,33,805,40,4,91.5,1,0,1
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,282,421,25,750.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,497,127,7,65,48,37,5,2703,806,32,379,311,138,325,9,3,700.0,1,0,1
318,492,136,5,76,50,94,12,5511,1511,39,897,451,875,313,381,20,875.0,0,0,0
319,475,126,3,61,43,52,6,1700,433,7,217,93,146,37,113,7,385.0,0,1,0
320,573,144,9,85,60,78,8,3198,857,97,470,420,332,1314,131,12,960.0,0,0,0


### Perform Best subset selection using Min MSE test error through Validation Set

In [5]:
X = df.drop(columns='Salary')
y = df['Salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=1)

In [6]:
# Process model on full X, y
def linear_model(feature_set, X, y):
    # Add Constant
    X_const = sm.add_constant(X[list(feature_set)])
    # Linear model
    lm = sm.OLS(y, X_const).fit()
    return lm

In [7]:
# Process subset using Validation set to return MSE info
def process_subset_validation_set(feature_set, X_train, X_test, y_train, y_test):
    # Create X_train, X_test
    X_train_const = sm.add_constant(X_train[list(feature_set)])
    X_test_const = sm.add_constant(X_test[list(feature_set)])

    # Fit model on feature_set and calculate RSS
    model = sm.OLS(y_train, X_train_const).fit()

    # Calculate MSE
    y_pred = model.predict(X_test_const)
    MSE = mean_squared_error(y_test, y_pred)

    return {"predictors": feature_set, "MSE": MSE}

In [8]:
def process_subset_validation_set_n_times(feature_set, X, y, n):
    
    MSE_results = []
    # Create X_train, X_test
    for i in range(n):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True)

        X_train_const = sm.add_constant(X_train[list(feature_set)])
        X_test_const = sm.add_constant(X_test[list(feature_set)])

        # Fit model on feature_set and calculate RSS
        model = sm.OLS(y_train, X_train_const).fit()

        # Calculate MSE
        y_pred = model.predict(X_test_const)
        MSE_results.append(mean_squared_error(y_test, y_pred))

    return {"predictors": feature_set, "MSE": np.mean(MSE_results)}

In [9]:
def process_subset_k_folds(feature_set, X, y, k):
    
    MSE_results = []
    # Crossvalidation with K-folds
    cv = KFold(n_splits=10, random_state=None, shuffle=False)
    for train_index, test_index in cv.split(X):

        X_train, y_train = X.iloc[train_index], y.iloc[train_index]
        X_test, y_test = X.iloc[test_index], y.iloc[test_index]

        X_train_const = sm.add_constant(X_train[list(feature_set)])
        X_test_const = sm.add_constant(X_test[list(feature_set)])

        # Fit model on feature_set and calculate RSS
        model = sm.OLS(y_train, X_train_const).fit()

        # Calculate MSE
        y_pred = model.predict(X_test_const)
        MSE_results.append(mean_squared_error(y_test, y_pred))
    
    return {"predictors": feature_set, "MSE": np.mean(MSE_results)}

In [10]:
def get_best_subset(number_predictors, cv='Validation_set'):
    # Counting time
    tic = time.time()
    predictors = X.columns
    results = []

    # Run for-loops to append results
    if cv=='K_folds':
        for feature_set in itertools.combinations(predictors, number_predictors):
            results.append(process_subset_k_folds(feature_set, X, y, k=10))        
    else:
        for feature_set in itertools.combinations(predictors, number_predictors):
            results.append(process_subset_validation_set_n_times(feature_set, X, y, n=10))

    # Wrap result in dataframe
    models = pd.DataFrame(results)

    # Choose the model with the lowest MSE
    best_model = models.loc[models['MSE'].argmin()]
    
    toc = time.time()
    print("Processed {:5} models on {:2} predictors in {:6.2f} seconds.".format(models.shape[0], number_predictors, (toc-tic)))
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [11]:
models_best_validation_set = pd.DataFrame(columns=["predictors", "MSE"])

tic = time.time()
for i in range(1,5):
    models_best_validation_set.loc[i] = get_best_subset(i, cv="Validation_set")

toc = time.time()
print("Total elapsed time: {:.2f} seconds.".format((toc-tic)))

Processed    19 models on  1 predictors in   0.86 seconds.
Processed   171 models on  2 predictors in   7.58 seconds.
Processed   969 models on  3 predictors in  48.59 seconds.
Processed  3876 models on  4 predictors in 195.64 seconds.
Total elapsed time: 252.68 seconds.


In [12]:
models_best_k_folds = pd.DataFrame(columns=["predictors", "MSE"])

tic = time.time()
for i in range(1,5):
    models_best_k_folds.loc[i] = get_best_subset(i, cv='K_folds')

toc = time.time()
print("Total elapsed time: {:.2f} seconds.".format((toc-tic)))

Processed    19 models on  1 predictors in   0.89 seconds.
Processed   171 models on  2 predictors in   7.95 seconds.
Processed   969 models on  3 predictors in  44.72 seconds.
Processed  3876 models on  4 predictors in 188.84 seconds.
Total elapsed time: 242.41 seconds.


In [13]:
models_best_validation_set

,predictors,MSE
1,"(CHits,)",134602.009258
2,"(Walks, CAtBat)",107341.443257
3,"(Hits, Years, CHmRun)",101112.434753
4,"(Runs, CRuns, PutOuts, League_N)",89866.881233


In [14]:
models_best_k_folds

,predictors,MSE
1,"(CRBI,)",142840.888022
2,"(Hits, CRBI)",124466.240992
3,"(Runs, CRBI, PutOuts)",119528.329256
4,"(Hits, CRBI, PutOuts, Division_W)",114839.706281


In [15]:
# Perform Forward Stepwise Selection
def forward_stepwise(predictors, cv="Validation_set"):
    
    # Pull out predictors we still need to process
    remaining_predictors = [p for p in X.columns if p not in predictors]

    tic = time.time()

    results = []

    if cv=='K_folds':
        for p in remaining_predictors:
            results.append(process_subset_k_folds(predictors + [p], X, y, k=10))
    else:
        for p in remaining_predictors:
            results.append(process_subset_validation_set_n_times(predictors + [p], X, y, n=10))

    # Wrap result up in a nice dataframe
    models = pd.DataFrame(results)

    # Choose model with the highest R2
    best_model = models.loc[models['MSE'].argmin()]

    toc = time.time()

    print('Processed {:3} models on {:2} predictors in {:6.2f} seconds.'.format(models.shape[0], len(predictors)+1, (toc-tic)))
    return best_model

In [16]:
models_fwd_validation_set = pd.DataFrame(columns=['predictors','MSE'])

tic = time.time()
predictors = []

for i in range(1, len(X.columns)+1):
    models_fwd_validation_set.loc[i] = forward_stepwise(predictors, cv='Validation_set')
    predictors = list(models_fwd_validation_set.loc[i,'predictors'])

toc = time.time()
print('Total elapsed time: {:.2f} seconds.'.format(toc-tic))

Processed  19 models on  1 predictors in   0.88 seconds.
Processed  18 models on  2 predictors in   0.88 seconds.
Processed  17 models on  3 predictors in   0.82 seconds.
Processed  16 models on  4 predictors in   0.80 seconds.
Processed  15 models on  5 predictors in   0.95 seconds.
Processed  14 models on  6 predictors in   0.91 seconds.
Processed  13 models on  7 predictors in   0.88 seconds.
Processed  12 models on  8 predictors in   0.91 seconds.
Processed  11 models on  9 predictors in   0.86 seconds.
Processed  10 models on 10 predictors in   0.82 seconds.
Processed   9 models on 11 predictors in   0.79 seconds.
Processed   8 models on 12 predictors in   0.75 seconds.
Processed   7 models on 13 predictors in   0.61 seconds.
Processed   6 models on 14 predictors in   0.53 seconds.
Processed   5 models on 15 predictors in   0.49 seconds.
Processed   4 models on 16 predictors in   0.40 seconds.
Processed   3 models on 17 predictors in   0.29 seconds.
Processed   2 models on 18 pred

In [17]:
models_fwd_k_folds = pd.DataFrame(columns=['predictors','MSE'])

tic = time.time()
predictors = []

for i in range(1, len(X.columns)+1):
    models_fwd_k_folds.loc[i] = forward_stepwise(predictors, cv='K_folds')
    predictors = list(models_fwd_k_folds.loc[i,'predictors'])

toc = time.time()
print('Total elapsed time: {:.2f} seconds.'.format(toc-tic))

Processed  19 models on  1 predictors in   0.84 seconds.
Processed  18 models on  2 predictors in   0.86 seconds.
Processed  17 models on  3 predictors in   0.83 seconds.
Processed  16 models on  4 predictors in   0.95 seconds.
Processed  15 models on  5 predictors in   0.93 seconds.
Processed  14 models on  6 predictors in   0.88 seconds.
Processed  13 models on  7 predictors in   0.87 seconds.
Processed  12 models on  8 predictors in   0.83 seconds.
Processed  11 models on  9 predictors in   0.79 seconds.
Processed  10 models on 10 predictors in   0.71 seconds.
Processed   9 models on 11 predictors in   0.72 seconds.
Processed   8 models on 12 predictors in   0.67 seconds.
Processed   7 models on 13 predictors in   0.58 seconds.
Processed   6 models on 14 predictors in   0.51 seconds.
Processed   5 models on 15 predictors in   0.43 seconds.
Processed   4 models on 16 predictors in   0.37 seconds.
Processed   3 models on 17 predictors in   0.27 seconds.
Processed   2 models on 18 pred

In [18]:
models_fwd_validation_set

,predictors,MSE
1,[RBI],145521.346714
2,"[RBI, CAtBat]",115736.924844
3,"[RBI, CAtBat, Hits]",108384.344684
4,"[RBI, CAtBat, Hits, NewLeague_N]",105592.042809
5,"[RBI, CAtBat, Hits, NewLeague_N, Walks]",92774.700043
6,"[RBI, CAtBat, Hits, NewLeague_N, Walks, CRuns]",105607.889171
7,"[RBI, CAtBat, Hits, NewLeague_N, Walks, CRuns,...",113619.303878
8,"[RBI, CAtBat, Hits, NewLeague_N, Walks, CRuns,...",110642.639631
9,"[RBI, CAtBat, Hits, NewLeague_N, Walks, CRuns,...",109138.190078
10,"[RBI, CAtBat, Hits, NewLeague_N, Walks, CRuns,...",118189.124296


In [19]:
models_fwd_k_folds

,predictors,MSE
1,[CRBI],142840.888022
2,"[CRBI, Hits]",124466.240992
3,"[CRBI, Hits, Division_W]",119615.547260
4,"[CRBI, Hits, Division_W, PutOuts]",114839.706281
5,"[CRBI, Hits, Division_W, PutOuts, AtBat]",112424.871657
6,"[CRBI, Hits, Division_W, PutOuts, AtBat, Walks]",108549.675150
7,"[CRBI, Hits, Division_W, PutOuts, AtBat, Walks...",108803.722368
8,"[CRBI, Hits, Division_W, PutOuts, AtBat, Walks...",108860.384940
9,"[CRBI, Hits, Division_W, PutOuts, AtBat, Walks...",106654.374078
10,"[CRBI, Hits, Division_W, PutOuts, AtBat, Walks...",107211.098023


In [20]:
# Perform Backward Stepwise selection
def backward_stepwise(predictors, i, cv='Validation_set'):
    
    tic = time.time()

    results = []

    if cv=='K_folds':
        for p in itertools.combinations(predictors, i):
            results.append(process_subset_k_folds(p, X, y, k=10))
    else:
        for p in itertools.combinations(predictors, i):
            results.append(process_subset_validation_set_n_times(p, X, y, n=10))

    models = pd.DataFrame(results)

    best_model = models.loc[models['MSE'].argmin()]

    toc = time.time()

    print('Processed {:3} models on {:2} predictors in {:5.2f} seconds.'.format(models.shape[0], i, (toc-tic)))
    
    return best_model

In [21]:
models_bwd_validation_set = pd.DataFrame(columns=['predictors','MSE'])

tic = time.time()
predictors = X.columns

for i in range(len(X.columns), 0, -1):
    models_bwd_validation_set.loc[i] = backward_stepwise(predictors, i, cv='Validation_set')
    predictors = list(models_bwd_validation_set.loc[i]['predictors'])

toc = time.time()
print('Total elapsed time: {:.2f} seconds.'.format(toc-tic))

Processed   1 models on 19 predictors in  0.12 seconds.
Processed  19 models on 18 predictors in  1.43 seconds.
Processed  18 models on 17 predictors in  1.29 seconds.
Processed  17 models on 16 predictors in  1.21 seconds.
Processed  16 models on 15 predictors in  1.10 seconds.
Processed  15 models on 14 predictors in  1.02 seconds.
Processed  14 models on 13 predictors in  0.92 seconds.
Processed  13 models on 12 predictors in  0.85 seconds.
Processed  12 models on 11 predictors in  0.75 seconds.
Processed  11 models on 10 predictors in  0.67 seconds.
Processed  10 models on  9 predictors in  0.61 seconds.
Processed   9 models on  8 predictors in  0.52 seconds.
Processed   8 models on  7 predictors in  0.45 seconds.
Processed   7 models on  6 predictors in  0.38 seconds.
Processed   6 models on  5 predictors in  0.33 seconds.
Processed   5 models on  4 predictors in  0.26 seconds.
Processed   4 models on  3 predictors in  0.19 seconds.
Processed   3 models on  2 predictors in  0.15 s

In [22]:
models_bwd_k_folds = pd.DataFrame(columns=['predictors','MSE'])

tic = time.time()
predictors = X.columns

for i in range(len(X.columns), 0, -1):
    models_bwd_k_folds.loc[i] = backward_stepwise(predictors, i, cv='K_folds')
    predictors = list(models_bwd_k_folds.loc[i]['predictors'])

toc = time.time()
print('Total elapsed time: {:.2f} seconds.'.format(toc-tic))

Processed   1 models on 19 predictors in  0.09 seconds.
Processed  19 models on 18 predictors in  1.39 seconds.
Processed  18 models on 17 predictors in  1.28 seconds.
Processed  17 models on 16 predictors in  1.19 seconds.
Processed  16 models on 15 predictors in  1.10 seconds.
Processed  15 models on 14 predictors in  1.01 seconds.
Processed  14 models on 13 predictors in  0.91 seconds.
Processed  13 models on 12 predictors in  0.83 seconds.
Processed  12 models on 11 predictors in  0.73 seconds.
Processed  11 models on 10 predictors in  0.66 seconds.
Processed  10 models on  9 predictors in  0.59 seconds.
Processed   9 models on  8 predictors in  0.51 seconds.
Processed   8 models on  7 predictors in  0.46 seconds.
Processed   7 models on  6 predictors in  0.39 seconds.
Processed   6 models on  5 predictors in  0.33 seconds.
Processed   5 models on  4 predictors in  0.25 seconds.
Processed   4 models on  3 predictors in  0.21 seconds.
Processed   3 models on  2 predictors in  0.14 s

In [23]:
models_bwd_validation_set

,predictors,MSE
19,"(AtBat, Hits, HmRun, Runs, RBI, Walks, Years, ...",122730.774470
18,"(AtBat, Hits, HmRun, Runs, RBI, Walks, Years, ...",112178.919557
17,"(AtBat, Hits, HmRun, RBI, Walks, Years, CAtBat...",103306.440714
16,"(AtBat, Hits, HmRun, RBI, Walks, Years, CAtBat...",99326.828925
15,"(AtBat, Hits, HmRun, RBI, Walks, Years, CHits,...",99905.988043
14,"(AtBat, Hits, HmRun, RBI, Walks, Years, CHits,...",99838.686522
13,"(AtBat, Hits, HmRun, RBI, Walks, Years, CHits,...",93689.193874
12,"(AtBat, Hits, HmRun, RBI, Walks, CHits, CRBI, ...",100999.442676
11,"(AtBat, Hits, HmRun, RBI, Walks, CRBI, CWalks,...",101193.635335
10,"(AtBat, Hits, HmRun, Walks, CRBI, CWalks, PutO...",101540.355674


In [24]:
models_bwd_k_folds

,predictors,MSE
19,"(AtBat, Hits, HmRun, Runs, RBI, Walks, Years, ...",116599.013674
18,"(AtBat, Hits, HmRun, Runs, RBI, Walks, Years, ...",114637.210275
17,"(AtBat, Hits, HmRun, Runs, RBI, Walks, Years, ...",112986.927921
16,"(AtBat, Hits, HmRun, Runs, Walks, Years, CAtBa...",111421.815540
15,"(AtBat, Hits, HmRun, Runs, Walks, CAtBat, CHmR...",110155.150976
14,"(AtBat, Hits, HmRun, Runs, Walks, CAtBat, CRun...",109272.004283
13,"(AtBat, Hits, HmRun, Runs, Walks, CAtBat, CRun...",108570.674432
12,"(AtBat, Hits, Runs, Walks, CAtBat, CRuns, CRBI...",107998.046793
11,"(AtBat, Hits, Walks, CAtBat, CRuns, CRBI, CWal...",107289.620703
10,"(AtBat, Hits, Walks, CAtBat, CRuns, CRBI, CWal...",106738.032067
